# Exemple. Titanic.

Primerament carregarem les dades i obtindrem una descripció general.

In [23]:
# Carreguem el fitxer titanic.csv

import pandas as pd
#from sklearn import datasets
#from sklearn.model_selection import KFold

titanic = pd.read_csv("./titanic.csv")
titanic.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Abans de continuar, anem a buscar valors perduts (NaN, ?, N/A, o cel·les en blanc) per evitar erros a l'hora d'entrenar models. En cas de contindre valors perduts, l'algorisme d'entrenament ens donarà un error del tipus *ValueError: Input contains NaN, infinity or a value too large for dtype('float64')*.

Per veure els nuls, anem a fer ús de la funció `isnull().any()` que comprovarà les característiques o columnes, i ens retornarà *False* si no falten valors o *True* si sí que en falten.

In [24]:
# Carreguem les dades en un nou dataframe amb les columnes amb les que anem a treballar
titanic2=titanic[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Survived']]
print (titanic2.isnull().any())

Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Survived    False
dtype: bool


Com veiem, falten valors a la columna Age. Anem a veure quins són aquests valors:

In [25]:
registers_amb_valors_perduts=pd.isnull(titanic2)
print(registers_amb_valors_perduts)

     Pclass    Sex    Age  SibSp  Parch  Survived
0     False  False  False  False  False     False
1     False  False  False  False  False     False
2     False  False  False  False  False     False
3     False  False  False  False  False     False
4     False  False  False  False  False     False
..      ...    ...    ...    ...    ...       ...
886   False  False  False  False  False     False
887   False  False  False  False  False     False
888   False  False   True  False  False     False
889   False  False  False  False  False     False
890   False  False  False  False  False     False

[891 rows x 6 columns]


Com veiem, açò ens retorna totes les files, indicant si hi ha o no nuls en cadascuna. Potser és més pràctic mirar en quines files estan aquests nuls. Per a això:

In [26]:
# Per obtindre les files on estan els nuls:
print(titanic2[titanic2.isnull().any(axis=1)])

     Pclass     Sex  Age  SibSp  Parch  Survived
5         3    male  NaN      0      0         0
17        2    male  NaN      0      0         1
19        3  female  NaN      0      0         1
26        3    male  NaN      0      0         0
28        3  female  NaN      0      0         1
..      ...     ...  ...    ...    ...       ...
859       3    male  NaN      0      0         0
863       3  female  NaN      8      2         0
868       3    male  NaN      0      0         0
878       3    male  NaN      0      0         0
888       3  female  NaN      1      2         0

[177 rows x 6 columns]


In [27]:

# I per obtindre les columnes amb nuls:
print(titanic2.columns[titanic2.isnull().any()])

Index(['Age'], dtype='object')


Una primera solució pot ser eliminar els registres fent ús de l'ordre `dropna` de Pandas. Eliminarem les files on la columna *Age* siga nul·la o buïda:

In [28]:
# Eliminem les files amb dades perdudes en una sola columna
# Fem ús d'axis=0 per eliminar files (amb axis=1 eliminariem columnes)
titanic2=titanic2.dropna(subset = ['Age'], axis = 0)
print(titanic2.isnull().any())
titanic2 # Ens quedaran 714 files per treballar

# Comprovem que no hi ha cap dada perduda
print ("\nHi ha dades perdudes? ",titanic2.isnull().any().any())

Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Survived    False
dtype: bool

Hi ha dades perdudes?  False


Una altr opció, havera estat reemplaçar els valors nuls per la mitjana. Per a això, en primer lloc ens creem altra còpia de les dades que ens interessen:

In [53]:
titanic3=titanic[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Survived']].copy() # Utilitzem copy per no fer un subset, sinò una còpia completa
print (titanic3.isnull().any())

Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Survived    False
dtype: bool


In [54]:
# Calculem la mitjana de la mitjana de l'edat

mitjana_edat = titanic3["Age"].mean()
mitjana_edat

29.69911764705882

In [55]:
# I afegim esta mitjana a la propietat Age d'aquelles cel·les en blanc

titanic3["Age"].fillna(mitjana_edat, inplace=True)
titanic3

,Pclass,Sex,Age,SibSp,Parch,Survived
0,3,male,22.000000,1,0,0
1,1,female,38.000000,1,0,1
2,3,female,26.000000,0,0,1
3,1,female,35.000000,1,0,1
4,3,male,35.000000,0,0,0
...,...,...,...,...,...,...
886,2,male,27.000000,0,0,0
887,1,female,19.000000,0,0,1
888,3,female,29.699118,1,2,0
889,1,male,26.000000,0,0,1


In [56]:
# I comprovem si queda algun nul
print(titanic3.isnull().any())

# Comprovem que no hi ha cap dada perduda
print ("\nHi ha dades perdudes? ",titanic3.isnull().any().any())

Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Survived    False
dtype: bool

Hi ha dades perdudes?  False
